In [7]:
import pandas as pd
import numpy as np
import powerlaw
from pathlib import Path
import glob

def analyze_degree_power_law(adjacency_matrix: np.ndarray) -> tuple[float, float] | None:
    """
    Computes the power-law exponent and a p-value for the network's degree distribution.

    Args:
        adjacency_matrix: A square NumPy array representing the network's connections.

    Returns:
        A tuple containing:
        - alpha (float): The power-law exponent.
        - p_value (float): The p-value from comparing the power-law fit against an
                           exponential distribution. It helps test if the power-law
                           is a significantly better model for the data.
        Returns None if the data is insufficient to fit.
    """
    if adjacency_matrix.shape[0] != adjacency_matrix.shape[1]:
        raise ValueError("Adjacency matrix must be square.")

    # Calculate the degree of each node (sum of connections per row)
    degrees = adjacency_matrix.sum(axis=1)

    # Filter out nodes with a degree of 0
    degrees = degrees[degrees > 0]
    
    if len(degrees) < 2:
        print("Not enough data with degree > 0 to fit a power law.")
        return None

    # Fit the power-law distribution to the degree data
    fit = powerlaw.Fit(degrees, discrete=True, verbose=False)

    # Get the power-law exponent (alpha)
    alpha = fit.power_law.alpha

    # Compare the power-law fit to an exponential distribution to get a p-value
    # R is the log-likelihood ratio; a positive R indicates the power-law is a better fit.
    R, p_value_exponential = fit.distribution_compare('power_law', 'exponential', normalized_ratio=True)
    R, p_value_lognormal = fit.distribution_compare('power_law', 'lognormal', normalized_ratio=True)

    return alpha, p_value_exponential, p_value_lognormal

In [8]:
import numpy as np
import matlab.engine

def compute_polar_coordinates(
    adj_matrix: np.ndarray,
    matlab_script_path: str,
    rad_type: str,
    pre_weighting: str,
    dim_red: str,
    angular_adjustment: str,
    dims: int = 2
) -> np.ndarray:
    """
    Computes polar coordinates for a network using a MATLAB embedding function.

    This function starts a MATLAB engine, checks if the input matrix is symmetric,
    calls the 'coalescent_embedding_v2_1' MATLAB script, and returns the
    resulting coordinates.

    Returns:
        np.ndarray:
            An (N, 2) array of polar coordinates [theta, r] for each node.
    """

    np.fill_diagonal(adj_matrix, 0)
    
    # --- 1. Ensure the matrix is symmetric ---
    if not np.allclose(adj_matrix, adj_matrix.T):
        print("DEBUG: Adjacency matrix was not symmetric. Forcing symmetry.")
        adj_matrix = np.maximum(adj_matrix, adj_matrix.T)

    print("Starting MATLAB engine...")
    eng = matlab.engine.start_matlab()
    print("MATLAB engine started successfully.")

    coords = None
    gamma = None
    try:
        # --- 2. Add the MATLAB script's directory to the MATLAB path ---
        eng.addpath(matlab_script_path, nargout=0)
        print(f"Added '{matlab_script_path}' to MATLAB path.")
        
        # Add matlab_bgl directory for compiled MEX files
        matlab_bgl_path = matlab_script_path + r'\matlab_bgl'
        eng.addpath(matlab_bgl_path, nargout=0)
        eng.addpath(eng.genpath(matlab_bgl_path), nargout=0)
        print(f"Added matlab_bgl and subdirectories to MATLAB path.")

        # --- 3. Convert the NumPy array to a MATLAB-compatible format ---
        matlab_adj_matrix = matlab.double(adj_matrix.tolist())

        # --- 4. Call the MATLAB function ---
        print("Calling MATLAB function 'coalescent_embedding_v2_1'...")
        # We expect 2 outputs (coords, gamma), so we set nargout=2
        matlab_coords, gamma = eng.coalescent_embedding_v2_1(
            matlab_adj_matrix,
            pre_weighting,
            dim_red,
            rad_type,
            angular_adjustment,
            dims,
            nargout=2
        )
        print("MATLAB function executed successfully.")

        # --- 5. Convert the result back to a NumPy array ---
        coords = np.array(matlab_coords)
        
    except Exception as e:
        print(f"An error occurred during MATLAB execution: {e}")
        raise
    finally:
        # --- 6. Ensure the MATLAB engine is always stopped ---
        print("Stopping MATLAB engine.")
        eng.quit()

    return coords, gamma

In [3]:
print("--- Starting MATLAB Engine from terminal ---")
try:
    eng = matlab.engine.start_matlab()
    print("✅ Engine started successfully.")

    # Test the Symbolic Math Toolbox
    result = eng.zeta(2.0)
    print(f"✅ Symbolic Math Toolbox found! zeta(2.0) = {result}")

    eng.quit()
    print("--- Test complete ---")
except Exception as e:
    print(f"❌ An error occurred: {e}")

--- Starting MATLAB Engine from terminal ---
✅ Engine started successfully.
✅ Symbolic Math Toolbox found! zeta(2.0) = 1.6449340668482264
--- Test complete ---


In [9]:
def plot_save_figure(coords, adj_matrix, matlab_script_path, filename):
    eng = None  # Initialize eng to None
    try:
        # --- 1. Start the MATLAB engine ---
        print("Starting MATLAB engine...")
        eng = matlab.engine.start_matlab()
        print("MATLAB engine started successfully.")

        # --- 2. Add the script's location to the MATLAB path ---
        eng.addpath(matlab_script_path, nargout=0)
        print(f"Added '{matlab_script_path}' to MATLAB path.")

        # --- 3. Extract and prepare data from reservoir_params ---
        coords = coords
        
        print(f"{coords=}")
        matlab_coords = matlab.double(coords.tolist())
        
        print(f"{adj_matrix=}")

        matlab_adj_matrix = matlab.double(adj_matrix.tolist())

        num_nodes = coords.shape[0]
        # Create a list of shapes, which will become a MATLAB cell array
        matlab_node_shapes = ['o'] * num_nodes

        save_filename = f"{filename}_figure.png"

        # --- 4. Call the MATLAB plotting function ---
        print("Calling MATLAB function 'plot_hyperbolic_network_v3'...")
        # Call the function and capture the figure handle in a variable 'h'
        eng.plot_hyperbolic_network_v3(
            matlab_adj_matrix,
            matlab_coords,
            'native',
            eng.cell(0),
            filename,
            matlab_node_shapes, # <-- Pass the list here
            nargout=0
        )
        h = eng.gcf(nargout=1)
        print("MATLAB function executed successfully.")
        # 1. Set the measurement units to inches for predictable sizing.
        eng.set(h, 'PaperUnits', 'inches', nargout=0)

        # 2. Force the figure's paper size to a standard 8x6 inches.
        #    The [0, 0] part is the position, [8, 6] is the width and height.
        eng.set(h, 'PaperPosition', matlab.double([0, 0, 8, 6]), nargout=0)

        # 3. Now, print the resized figure with a defined resolution.
        print(f"Saving resized figure to {save_filename}...")
        eng.print(h, save_filename, '-dpng', '-r150', nargout=0)
        print("Figure saved successfully.")

        print("Figure saved successfully.")

    except Exception as e:
        print(f"An error occurred during MATLAB execution: {e}")
    finally:
        # --- 6. Ensure the MATLAB engine is always stopped ---
        if eng:
            print("Stopping MATLAB engine.")
            eng.quit()

In [10]:
# TODO
# check the different parameters to visualise the networks, see what works
'''
%%% INPUT %%%
%
% pre_weighting - rule for pre-weighting the matrix, the alternatives are:
%   'original' -> the original weights are considered;
%                 NB: they should suggest distances and not similarities
%   'reverse'  -> the original weights reversed are considered;
%                 NB: to use when they suggest similarities
%   'RA1'      -> Repulsion-Attraction v1
%   'RA2'      -> Repulsion-Attraction v2
%   'EBC'      -> Edge-Betweenness-Centrality (requires MEX function)
%   'nEBC'      -> normalized Edge-Betweenness-Centrality (requires MEX function)
%
% dim_red - dimension reduction technique, the alternatives are:
%   'ISO'   -> Isomap (valid for 2D and 3D)
%   'ncISO' -> noncentered Isomap (valid for 2D and 3D)
%   'LE'    -> Laplacian Eigenmaps (valid for 2D and 3D)
%   'MCE'   -> Minimum Curvilinear Embedding (only valid for 2D)
%   'ncMCE' -> noncentered Minimum Curvilinear Embedding (only valid for 2D)
%
% rad_type - radial coordinates method, the alternatives are:
%   'degree' -> node degree
%   'pre_we_type1' -> node strength based on the pre-weighting methods 
%   'pre_we_type2' -> node degree is used for the inference of gamma value and node strength for the sorting of the nodes
%
% angular_adjustment - method for the angular adjustment, the alternatives are:
%   'original' -> original angular distances are preserved (valid for 2D and 3D)
%   'EA'       -> equidistant adjustment (only valid for 2D)
%   
% dims - dimensions of the hyperbolic embedding space, the alternatives are:
%   2 -> hyperbolic disk
%   3 -> hyperbolic sphere
'''
## DEFAULT PARAMETERS
rad = 'degree'
pre_w = 'EBC'  # Changed from 'EBC' to avoid MEX function requirement
dim_r = 'ncISO'
ang_adj = 'EA'

print(f"Using parameters: rad={rad}, pre_w={pre_w}, dim_r={dim_r}, ang_adj={ang_adj}")

Using parameters: rad=degree, pre_w=EBC, dim_r=ncISO, ang_adj=EA


In [11]:
import numpy as np
from pathlib import Path

matlab_ce_path = r"D:\Tsinghua Classes and Papers\Network Science\migra-net-china\matlab\plot_hyperbolic_network_v2\matlab_scripts"

# Path to network data folder
network_data_path = Path(r"D:\Tsinghua Classes and Papers\Network Science\migra-net-china\src\notebooks\network_data")

# Specify which year to process
YEAR = 2010

# Load the specific network file
filepath = network_data_path / f"network_{YEAR}_weighted_directed.npy"

print(f"Processing network for year {YEAR}")
print(f"Loading: {filepath}")

# Load the weighted directed network
adj_weighted_directed = np.load(filepath)
print(f"Loaded: {adj_weighted_directed.shape}")

# Convert to unweighted (binary): any edge > 0 becomes 1
adj_unweighted_directed = (adj_weighted_directed > 0).astype(int)

# Convert to undirected (symmetrize by taking maximum)
adj_matrix = np.maximum(adj_unweighted_directed, adj_unweighted_directed.T)

num_nodes = adj_matrix.shape[0]
num_edges = np.count_nonzero(adj_matrix) // 2
print(f"Converted to unweighted undirected: {num_nodes} nodes, {num_edges} edges")

# Compute hyperbolic coordinates
print("\nComputing hyperbolic coordinates...")
coords, gamma = compute_polar_coordinates(
    adj_matrix, matlab_ce_path, 
    rad_type=rad, pre_weighting=pre_w, 
    dim_red=dim_r, angular_adjustment=ang_adj
)

# Plot and save figure
print("\nCreating visualization...")
plot_save_figure(coords, adj_matrix, matlab_ce_path, f"network_{YEAR}")

# Analyze degree distribution
print("\nAnalyzing degree distribution...")
alpha, p_value_exponential, p_value_lognormal = analyze_degree_power_law(adj_matrix)

print(f"\n{'='*60}")
print(f"Results for year {YEAR}:")
print(f"  Nodes: {num_nodes}")
print(f"  Edges: {num_edges}")
print(f"  Alpha (degree exponent): {alpha:.3f}")
print(f"  P-value (vs exponential): {p_value_exponential:.4f}")
print(f"  P-value (vs lognormal): {p_value_lognormal:.4f}")
print(f"  Gamma (hyperbolic): {gamma:.3f}")
print(f"{'='*60}")

Processing network for year 2010
Loading: D:\Tsinghua Classes and Papers\Network Science\migra-net-china\src\notebooks\network_data\network_2010_weighted_directed.npy
Loaded: (377, 377)
Converted to unweighted undirected: 377 nodes, 4101 edges

Computing hyperbolic coordinates...
Starting MATLAB engine...
MATLAB engine started successfully.
Added 'D:\Tsinghua Classes and Papers\Network Science\migra-net-china\matlab\plot_hyperbolic_network_v2\matlab_scripts' to MATLAB path.
Added matlab_bgl and subdirectories to MATLAB path.
Calling MATLAB function 'coalescent_embedding_v2_1'...
MATLAB function executed successfully.
Stopping MATLAB engine.

Creating visualization...
Starting MATLAB engine...
MATLAB engine started successfully.
Added 'D:\Tsinghua Classes and Papers\Network Science\migra-net-china\matlab\plot_hyperbolic_network_v2\matlab_scripts' to MATLAB path.
coords=array([[ 5.11654613,  9.72790953],
       [ 3.16659206,  8.32652301],
       [ 2.88326541,  6.04083628],
       [ 0.866